In [151]:
import pandas as pd
from tqdm import tqdm

In [119]:
df = pd.read_csv('Data/010/trajectories.csv', 
                 names=['Latitude', 'Longitude', '0', 'Altitude', 'Date', 'Date_str', 'Time_str'],
                parse_dates=True)

In [120]:
df['DateTime'] = df['Date_str'] + ' ' + df['Time_str']
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [121]:
df = df.drop(columns=['Date_str', 'Time_str', '0', 'Date'])

In [97]:
labels = pd.read_csv('Data/010/labels.txt', delim_whitespace=True, skiprows=1, header=None)

In [98]:
labels

,0,1,2,3,4
0,2007/06/26,11:32:29,2007/06/26,11:40:29,bus
1,2008/03/28,14:52:54,2008/03/28,15:59:59,train
2,2008/03/28,16:00:00,2008/03/28,22:02:00,train
3,2008/03/29,01:27:50,2008/03/29,15:59:59,train
4,2008/03/29,16:00:00,2008/03/30,15:59:59,train
...,...,...,...,...,...
429,2008/12/07,10:30:54,2008/12/07,10:34:14,taxi
430,2008/12/07,10:59:29,2008/12/07,11:29:48,train
431,2008/12/07,11:43:12,2008/12/07,12:23:26,bus
432,2008/12/07,12:23:34,2008/12/07,12:25:07,walk


In [99]:
labels['StartTime'] = labels[0] + ' ' + labels[1]
labels['EndTime'] = labels[2] + ' ' + labels[3]
labels['StartTime'] = pd.to_datetime(labels['StartTime'])
labels['EndTime'] = pd.to_datetime(labels['EndTime'])
labels = labels.drop(columns=[0, 1, 2, 3])
labels = labels.rename(columns={4: "TransportMode"})

In [118]:
labels

,TransportMode,StartTime,EndTime
0,bus,2007-06-26 11:32:29,2007-06-26 11:40:29
1,train,2008-03-28 14:52:54,2008-03-28 15:59:59
2,train,2008-03-28 16:00:00,2008-03-28 22:02:00
3,train,2008-03-29 01:27:50,2008-03-29 15:59:59
4,train,2008-03-29 16:00:00,2008-03-30 15:59:59
...,...,...,...
429,taxi,2008-12-07 10:30:54,2008-12-07 10:34:14
430,train,2008-12-07 10:59:29,2008-12-07 11:29:48
431,bus,2008-12-07 11:43:12,2008-12-07 12:23:26
432,walk,2008-12-07 12:23:34,2008-12-07 12:25:07


In [ ]:
target = pd.DataFrame()
for i in tqdm(range(len(labels.index))):
    mask = (df['DateTime'] >= labels.loc[index, 'StartTime']) & (df['DateTime'] <= labels.loc[index, 'EndTime'])
    trip_data = df.loc[mask]
    trip_data = trip_data.reset_index()
    for index_trip, row_trip in trip_data.iterrows():
        if index_trip != len(trip_data.index) - 1:
            pd.concat([target, pd.DataFrame({'LatStart': trip_data.loc[index_trip, 'Latitude'],
                                    'LongStart': trip_data.loc[index_trip, 'Longitude'],
                                    'AltitudeStart': trip_data.loc[index_trip, 'Altitude'],
                                    'TimeStart': trip_data.loc[index_trip, 'DateTime'],
                                    'LatEnd': trip_data.loc[index_trip+1, 'Latitude'],
                                    'LongEnd': trip_data.loc[index_trip+1, 'Longitude'],
                                    'AltitudeEnd': trip_data.loc[index_trip+1, 'Altitude'],
                                    'TimeEnd': trip_data.loc[index_trip+1, 'DateTime'],
                                    'TransportMode': labels.loc[index, 'TransportMode']},  index=[0])],
                     ignore_index=True)
#             target = target.concat({'LatStart': trip_data.loc[index_trip, 'Latitude'],
#                                     'LongStart': trip_data.loc[index_trip, 'Longitude'],
#                                     'AltitudeStart': trip_data.loc[index_trip, 'Altitude'],
#                                     'TimeStart': trip_data.loc[index_trip, 'DateTime'],
#                                     'LatEnd': trip_data.loc[index_trip+1, 'Latitude'],
#                                     'LongEnd': trip_data.loc[index_trip+1, 'Longitude'],
#                                     'AltitudeEnd': trip_data.loc[index_trip+1, 'Altitude'],
#                                     'TimeEnd': trip_data.loc[index_trip+1, 'DateTime'],
#                                     'TransportMode': labels.loc[index, 'TransportMode']},
#                                     ignore_index=True)
    
#     df.loc[mask, 'TransportMode'] = labels.loc[index, 'TransportMode']
    

  2%|█▉                                                                                                                       | 7/434 [02:10<2:11:28, 18.48s/it]

In [142]:
target

,LatStart,LongStart,AltitudeStart,TimeStart,LatEnd,LongEnd,AltitudeEnd,TimeEnd
0,39.894178,116.318200,-777,2008-03-28 14:54:40,39.894505,116.321132,-777,2008-03-28 14:55:14
1,39.894505,116.321132,-777,2008-03-28 14:55:14,39.894953,116.326452,-777,2008-03-28 14:56:13
2,39.894953,116.326452,-777,2008-03-28 14:56:13,39.894600,116.332542,-777,2008-03-28 14:57:12
3,39.894600,116.332542,-777,2008-03-28 14:57:12,39.889622,116.337040,-777,2008-03-28 14:58:11
4,39.889622,116.337040,-777,2008-03-28 14:58:11,39.882090,116.338353,-777,2008-03-28 14:59:10
...,...,...,...,...,...,...,...,...
63,39.511005,116.700307,-777,2008-03-28 15:55:05,39.511005,116.700307,-777,2008-03-28 15:56:02
64,39.511005,116.700307,-777,2008-03-28 15:56:02,39.511005,116.700307,-777,2008-03-28 15:56:04
65,39.511005,116.700307,-777,2008-03-28 15:56:04,39.510970,116.700392,-777,2008-03-28 15:57:03
66,39.510970,116.700392,-777,2008-03-28 15:57:03,39.509652,116.702593,-777,2008-03-28 15:58:03


In [115]:
print(df['TransportMode'].isna().sum())
len(df.index)

401412


935528

In [116]:
df = df.dropna(subset=['TransportMode'])

In [117]:
df

,Latitude,Longitude,Altitude,DateTime,TransportMode
180180,39.894178,116.318200,-777,2008-03-28 14:54:40,train
180181,39.894505,116.321132,-777,2008-03-28 14:55:14,train
180182,39.894953,116.326452,-777,2008-03-28 14:56:13,train
180183,39.894600,116.332542,-777,2008-03-28 14:57:12,train
180184,39.889622,116.337040,-777,2008-03-28 14:58:11,train
...,...,...,...,...,...
719969,39.991644,116.326376,184,2008-12-07 12:37:24,taxi
719970,39.991641,116.326325,180,2008-12-07 12:37:25,taxi
719971,39.991641,116.326289,177,2008-12-07 12:37:26,taxi
719972,39.991643,116.326273,174,2008-12-07 12:37:27,taxi
